In [ ]:
import pandas as pd

### The way it was in the example from Dimi (using neo4j-driver):

In [ ]:
# pip install neo4j-driver

from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
  "bolt://localhost:7687",
  auth=basic_auth("neo4j", "0306"))

cypher_query = '''
MATCH (a:Officer {name:$name})-[r:officer_of|intermediary_of|registered_address*..10]-(b)
RETURN b.name as name, b.node_id as node_id LIMIT 20
'''

with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(cypher_query,
                      name="Ross, Jr. - Wilbur Louis").data())
  for record in results:
    print(record)
  df = pd.DataFrame().from_records(results)

driver.close()
df.head()

### If you want to use the graphdatascience client instead: 
You can find more information about the gds python client itself and its methods etc. [here](https://neo4j.com/docs/graph-data-science-client/current/graph-object/).
General tutorials on the Graph Data Science plugin can be found [here](https://neo4j.com/docs/graph-data-science-client/current/graph-object/).

In [ ]:
# pip install graphdatascience
from graphdatascience import GraphDataScience

# Use Neo4j URI and credentials according to your setup
# NEO4J_URI could look similar to "bolt://my-server.neo4j.io:7687"
gds = GraphDataScience("bolt://localhost:7687", auth=("neo4j", "0306"))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

In [ ]:
# a simple cypher query:

degree_df = gds.run_cypher("""
MATCH (c)
RETURN c.name AS character,
       size((c)--()) AS degree, labels(c) as type
LIMIT 10
""")

degree_df.head() #did return a pandas df 

In [ ]:
# Project the graph into the GDS Graph Catalog
# We call the object representing the projected graph `G`
# note that you can't create a graph projection which already exists. The graph catalog is the same, i.e. shared, as the one you can access within the neo4j browser 
G, project_result = gds.graph.project("native-proj","*", {"link": {"type": '*', "orientation": 'UNDIRECTED'}})

# list objects in current graph catalog ("native-proj" should be there now)
gds.graph.list()

In [ ]:
# some numbers about our graph
print("Number of edges: ", G.relationship_count())
print("Number of nodes: ", G.node_count())
print("Graph density: ", G.density())

In [ ]:
G.drop() # removes the graph from the graph catalog
gds.close()